In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iterated-12preds/iterated_12vars.csv
/kaggle/input/dataset-lowest-aic452022/dataset_lowestAIC_longlat.csv
/kaggle/input/datasetlowestaic/dataset_lowestAIC.csv


In [2]:
#import dataset 
df = pd.read_csv(r'../input/dataset-lowest-aic452022/dataset_lowestAIC_longlat.csv', sep=';')

In [3]:
#examine
df.head()

,Unnamed: 0,Longitude,Latitude,mean_value_NO2,nightlight_450,nightlight_3150,population_1000,population_3000,road_class_2_100,road_class_2_25,road_class_2_50,road_class_3_3000,road_class_3_300,trop_mean_filt_2019,BldDen100,trafBuf100
0,0,10.008293,48.397079,29.250500,22.585377,13.402575,8123.319336,54316.64453,0.000000,0.000000,0.000000,203502.42190,4167.064941,0.000058,1.000663,0.000000
1,1,10.021227,53.523262,32.098936,23.546194,22.862799,9558.207031,76786.28125,0.000000,0.000000,0.000000,99596.04688,1850.278076,0.000081,0.978245,0.000000
2,2,10.053859,53.592301,58.703744,18.290140,16.493486,10471.541020,88439.60938,270.224976,46.359188,108.745453,265563.71880,2016.838379,0.000076,0.914638,674.791667
3,3,10.081685,53.529020,24.249276,23.645823,18.248226,11165.945310,76961.00000,0.000000,0.000000,0.000000,97921.56250,478.363281,0.000074,0.712983,0.000000
4,4,10.096292,48.847917,19.912116,14.026642,5.514785,6017.511230,30814.58984,0.000000,0.000000,0.000000,176265.68750,3338.221191,0.000047,1.000641,0.000000


In [4]:
#remove unique idenifier & geodata
df = df.drop(['Unnamed: 0', 'Longitude', 'Latitude'], axis=1)

In [5]:
y = df['mean_value_NO2']
x = df.drop(['mean_value_NO2'], axis=1)
feature_list = list(x.columns)

In [6]:
                       ## STEP 15. CONSTRUCTING RANDOM FOREST 
                       ## Import the model we are using
        
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


##STEP 8B. GENERATING CROSS VALIDATION (80% TRAINING OF MODEL; 
        ##20% TESTING OF MODEL;EVALUATING MODEL PERFORMANCE)

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25, random_state=20)
#random state is needed to ensure that same results are generated each time.

In [7]:
## STEP 16.ENSURING THAT X- AND Y-TRAINING SET CONTAIN SAME AMOUNT OF ROWS. SAME FOR TESTING.
print('Training Features Shape:',X_train.shape)
print('Training Labels Shape:', Y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)

Training Features Shape: (361, 12)
Training Labels Shape: (361,)
Testing Features Shape: (121, 12)
Testing Labels Shape: (121,)


In [8]:
## STEP 17. PARAMETERS FOR MODEL - RANDOM FOREST

# # Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, Y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [9]:
##OUT OF BAG SCORE
##SOURCE: https://gdcoder.com/random-forest-regression-model-explained-in-depth-part-2-python-code-snippet-using-sklearn/
m = RandomForestRegressor(n_estimators=1000, oob_score=True)
m.fit(X_train, Y_train)

print(m.oob_score_) # R^2 score

0.6979769396086769


In [10]:
##STEP 18. DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES (FOR TESTING DATASET)
expected_y  = Y_test
predicted_y = rf.predict(X_test) #Predicting the test set results using the random forest regressor model

In [11]:
##STEP 18. DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES (FOR TRAINING DATASET)

expected_y_tr = Y_train
predicted_y_tr = rf.predict(X_train)
# print(expected_y_tr, predicted_y_tr)

In [12]:
##STEP 12A. RMSE - TRAIN

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print("training rmse: ", rmse(predicted_y_tr, expected_y_tr))
print("testing rmse: ", rmse(predicted_y, Y_test))

training rmse:  2.532751898489329
testing rmse:  8.07256435198613


In [13]:
##STEP 12B. R2

from sklearn.metrics import r2_score
print("R2 score training data: ", r2_score(expected_y_tr, predicted_y_tr))
print("R2 score testing data: ", r2_score(Y_test, predicted_y))

R2 score training data:  0.9595342812428931
R2 score testing data:  0.6438696999883249


In [14]:
##STEP 12C. MEAN ABSOLUTE ERROR 
from sklearn.metrics import mean_absolute_error
print("MAE training data: ", mean_absolute_error(expected_y_tr, predicted_y_tr))
print("MAE testing data: ", mean_absolute_error(expected_y, predicted_y))

MAE training data:  1.7586397967329648
MAE testing data:  5.706397591193469


In [15]:
#STEP 10. HYPERPARAMETER TUNING
#STEP 10A. VERIFYING CURRENT RANDOM FOREST HYPERPARAMETERS 

from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [16]:
#STEP 10B. CREATE PARAMETER GRID

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [17]:
##STEP 10C. RANDOM SEARCH TRAINING (https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.4min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [18]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [19]:
## STEP 17. PARAMETERS FOR MODEL - RANDOM FOREST

# # Instantiate model with 1000 decision trees
rf_ran = RandomForestRegressor(n_estimators = 1000, random_state = 42, min_samples_split=2,
                          min_samples_leaf=1,max_features='sqrt',max_depth=20, bootstrap=True )
# Train the model on training data
rf_ran.fit(X_train, Y_train)

RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=1000,
                      random_state=42)

In [20]:
##STEP 18. DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES (FOR TESTING DATASET)
expected_yran  = Y_test
predicted_yran = rf_ran.predict(X_test) #Predicting the test set results using the random forest regressor model

In [21]:
##STEP 18. DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES (FOR TRAINING DATASET)

expected_y_tr_ran = Y_train
predicted_y_tr_ran = rf_ran.predict(X_train)
# print(expected_y_tr, predicted_y_tr)

In [22]:
##STEP 12A. RMSE - TRAIN

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print("training rmse: ", rmse(predicted_y_tr_ran, expected_y_tr_ran))
print("testing rmse: ", rmse(predicted_yran, Y_test))

training rmse:  2.5233467181208504
testing rmse:  7.904981257374466


In [23]:
##STEP 12B. R2

from sklearn.metrics import r2_score
print("R2 score training data: ", r2_score(expected_y_tr_ran, predicted_y_tr_ran))
print("R2 score testing data: ", r2_score(Y_test, predicted_yran))

R2 score training data:  0.9598342559406232
R2 score testing data:  0.6585024571695024


In [24]:
##STEP 12C. MEAN ABSOLUTE ERROR 
from sklearn.metrics import mean_absolute_error
print("MAE training data: ", mean_absolute_error(expected_y_tr_ran, predicted_y_tr_ran))
print("MAE testing data: ", mean_absolute_error(expected_yran, predicted_yran))

MAE training data:  1.785888887617458
MAE testing data:  5.579556901278602


In [25]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [26]:
# Fit the grid search to the data
grid_search.fit(X_train, Y_train)
grid_search.best_params_


Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  4.2min finished


{'bootstrap': True,
 'max_depth': 100,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

In [27]:
## STEP 17. PARAMETERS FOR MODEL - RANDOM FOREST

# # Instantiate model with 1000 decision trees
rf_grid = RandomForestRegressor(n_estimators = 100, random_state = 42, min_samples_split=10,
                          min_samples_leaf=3,max_features=2,max_depth=90, bootstrap=True )
# Train the model on training data
rf_grid.fit(X_train, Y_train)

RandomForestRegressor(max_depth=90, max_features=2, min_samples_leaf=3,
                      min_samples_split=10, random_state=42)

In [28]:
##STEP 18. DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES (FOR TESTING DATASET)
expected_ygrid  = Y_test
predicted_ygrid = rf_grid.predict(X_test) #Predicting the test set results using the random forest regressor model

In [29]:
##STEP 18. DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES (FOR TRAINING DATASET)

expected_y_tr_grid = Y_train
predicted_y_tr_grid = rf_grid.predict(X_train)
# print(expected_y_tr, predicted_y_tr)

In [30]:
##STEP 12A. RMSE - TRAIN

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print("training rmse: ", rmse(predicted_y_tr_grid, expected_y_tr_grid))
print("testing rmse: ", rmse(predicted_ygrid, Y_test))

training rmse:  4.892580580733285
testing rmse:  7.959163730995015


In [31]:
##STEP 12B. R2

from sklearn.metrics import r2_score
print("R2 score training data: ", r2_score(expected_y_tr_grid, predicted_y_tr_grid))
print("R2 score testing data: ", r2_score(Y_test, predicted_ygrid))

R2 score training data:  0.8489996569006868
R2 score testing data:  0.6538050155357024


In [32]:
##STEP 12C. MEAN ABSOLUTE ERROR 
from sklearn.metrics import mean_absolute_error
print("MAE training data: ", mean_absolute_error(expected_y_tr_grid, predicted_y_tr_grid))
print("MAE testing data: ", mean_absolute_error(expected_ygrid, predicted_ygrid))

MAE training data:  3.440899448264669
MAE testing data:  5.679518669197806


In [33]:
##RANDOM FOREST VISUALIZATIONS

# # Import tools needed for visualization
# from sklearn.tree import export_graphviz
# import pydot

# # Limit depth of tree to 3 levels
# rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
# rf_small.fit(X_train, Y_train)
# # Extract the small tree
# tree_small = rf_small.estimators_[5]
# # Save the tree as a png image
# export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# (graph, ) = pydot.graph_from_dot_file('small_tree.dot')
# graph.write_png('small_tree.png');